# Fleet Analytics & Prediction Tasks

## Classification Task
Predict whether a trip is operationally risky or problematic.

We define a trip as **High Risk (1)** if the status indicates a delay or problem
(e.g., "Delayed", "Cancelled", "Failed"). Otherwise, it is **Low Risk (0)**.

This helps the fleet manager identify trips likely to cause service issues.

## Regression Task
Predict the **maintenance_cost** of a vehicle/trip based on:
- distance_km
- vehicle age / type
- driving behaviour (violations, speeding incidents)
- fuel_cost, toll_cost, load_value
- weather_condition and route information

In [1]:
import pandas as pd

df = pd.read_csv("../data/fleet_dummy_5000.csv")

df.head()

,trip_id,driver_id,driver_name,origin,destination,distance_km,pickup_time,delivery_time,actual_delivery_time,status,...,profit_margin,vehicle_id,vehicle_type,violation_count,speeding_incidents,gps_start_lat,gps_start_lon,gps_end_lat,gps_end_lon,weather_condition
0,1,112,Driver_112,New York,Boston,615.26,2025-08-28 08:03:12.970230,2025-09-01 07:03:12.970230,2025-09-01 06:40:12.970230,Delivered,...,608.22,209,Truck,0,0,29.893102,-91.745548,26.687392,-113.888335,Sunny
1,2,139,Driver_139,Dallas,Atlanta,2112.03,2025-08-28 20:03:12.970302,2025-09-01 20:03:12.970302,2025-09-01 22:11:12.970302,Delivered,...,2764.47,211,Van,3,3,29.475943,-103.372264,33.766738,-98.863483,Cloudy
2,3,111,Driver_111,Miami,Boston,281.87,2025-08-27 05:03:12.970348,2025-08-28 22:03:12.970348,2025-08-28 21:54:12.970348,Scheduled,...,97.58,208,Van,0,0,25.038389,-74.604847,32.415163,-81.995082,Cloudy
3,4,113,Driver_113,Seattle,Los Angeles,1362.60,2025-08-28 21:03:12.970392,2025-09-01 03:03:12.970392,2025-09-01 05:22:12.970392,Delivered,...,1105.35,237,Van,1,1,35.949256,-92.280003,27.238830,-99.362534,Sunny
4,5,109,Driver_109,Seattle,Denver,410.36,2025-08-28 11:03:12.970434,2025-08-30 11:03:12.970434,2025-08-30 11:51:12.970434,Delivered,...,-32.84,213,Van,0,0,25.305279,-95.553751,34.761454,-76.599042,Rain


In [4]:
df["status"].value_counts()

status
Delivered     2973
In Transit    1013
Scheduled      513
Delayed        501
Name: count, dtype: int64

In [5]:
# Normalize status to lowercase
df["status_lower"] = df["status"].str.lower()

# Define high risk trips (only Delayed)
df["high_risk"] = (df["status_lower"] == "delayed").astype(int)

# Check result
df["high_risk"].value_counts()

high_risk
0    4499
1     501
Name: count, dtype: int64

In [6]:
# Convert pickup_time to datetime
df["pickup_time"] = pd.to_datetime(df["pickup_time"])

# Add time-based features
df["pickup_hour"] = df["pickup_time"].dt.hour
df["pickup_dayofweek"] = df["pickup_time"].dt.dayofweek

# Feature columns
features = [
    "distance_km",
    "fuel_cost",
    "driver_pay",
    "toll_cost",
    "load_value",
    "profit_margin",
    "violation_count",
    "speeding_incidents",
    "gps_start_lat",
    "gps_start_lon",
    "gps_end_lat",
    "gps_end_lon",
    "pickup_hour",
    "pickup_dayofweek"
]

# Feature matrix
X = df[features]

# Targets
y_class = df["high_risk"]          # classification
y_reg = df["maintenance_cost"]     # regression

X.head(), y_class.head(), y_reg.head()

(   distance_km  fuel_cost  driver_pay  toll_cost  load_value  profit_margin  \
 0       615.26     205.54      324.76      71.01     1292.17         608.22   
 1      2112.03     917.13     1050.47      41.84     4846.48        2764.47   
 2       281.87      76.47      161.41      67.92      676.83          97.58   
 3      1362.60     580.53      816.53      71.46     2735.60        1105.35   
 4       410.36     192.53      218.25      55.18      728.79         -32.84   
 
    violation_count  speeding_incidents  gps_start_lat  gps_start_lon  \
 0                0                   0      29.893102     -91.745548   
 1                3                   3      29.475943    -103.372264   
 2                0                   0      25.038389     -74.604847   
 3                1                   1      35.949256     -92.280003   
 4                0                   0      25.305279     -95.553751   
 
    gps_end_lat  gps_end_lon  pickup_hour  pickup_dayofweek  
 0    26.687392 

In [7]:
print("X shape:", X.shape)
print("y_class shape:", y_class.shape)
print("y_reg shape:", y_reg.shape)

df[features + ["maintenance_cost", "high_risk"]].isna().sum()

X shape: (5000, 14)
y_class shape: (5000,)
y_reg shape: (5000,)


distance_km           0
fuel_cost             0
driver_pay            0
toll_cost             0
load_value            0
profit_margin         0
violation_count       0
speeding_incidents    0
gps_start_lat         0
gps_start_lon         0
gps_end_lat           0
gps_end_lon           0
pickup_hour           0
pickup_dayofweek      0
maintenance_cost      0
high_risk             0
dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

# Classification split
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Regression split
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X, y_reg, test_size=0.2, random_state=42
)

print("Classification Train:", X_train_c.shape, "Test:", X_test_c.shape)
print("Regression Train:", X_train_r.shape, "Test:", X_test_r.shape)

Classification Train: (4000, 14) Test: (1000, 14)
Regression Train: (4000, 14) Test: (1000, 14)


In [9]:
numeric_features = X.columns.tolist()
numeric_features

['distance_km',
 'fuel_cost',
 'driver_pay',
 'toll_cost',
 'load_value',
 'profit_margin',
 'violation_count',
 'speeding_incidents',
 'gps_start_lat',
 'gps_start_lon',
 'gps_end_lat',
 'gps_end_lon',
 'pickup_hour',
 'pickup_dayofweek']

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Preprocessing pipeline for numeric features
preprocess_numeric = Pipeline(
    steps=[
        ("scaler", StandardScaler())
    ]
)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   trip_id               5000 non-null   int64  
 1   driver_id             5000 non-null   int64  
 2   driver_name           5000 non-null   object 
 3   origin                5000 non-null   object 
 4   destination           5000 non-null   object 
 5   distance_km           5000 non-null   float64
 6   pickup_time           5000 non-null   object 
 7   delivery_time         5000 non-null   object 
 8   actual_delivery_time  5000 non-null   object 
 9   status                5000 non-null   object 
 10  fuel_cost             5000 non-null   float64
 11  driver_pay            5000 non-null   float64
 12  maintenance_cost      5000 non-null   float64
 13  toll_cost             5000 non-null   float64
 14  load_value            5000 non-null   float64
 15  profit_margin        

In [3]:
df.describe()

,trip_id,driver_id,distance_km,fuel_cost,driver_pay,maintenance_cost,toll_cost,load_value,profit_margin,vehicle_id,violation_count,speeding_incidents,gps_start_lat,gps_start_lon,gps_end_lat,gps_end_lon
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,2500.500000,125.391600,1356.902714,554.283932,611.457656,176.087792,55.517052,2721.313518,1323.967086,225.472600,0.406000,0.406000,33.551597,-94.935752,33.415294,-94.714018
std,1443.520003,14.513786,666.350404,300.998545,327.766767,71.855372,26.546452,1410.386234,902.287093,14.509781,0.715309,0.715309,4.937381,14.517723,4.945796,14.360470
min,1.000000,101.000000,200.010000,52.580000,62.330000,50.020000,10.000000,320.880000,-296.690000,201.000000,0.000000,0.000000,25.003645,-119.993053,25.001161,-119.988266
25%,1250.750000,113.000000,778.557500,302.867500,336.092500,115.137500,32.097500,1521.575000,583.032500,213.000000,0.000000,0.000000,29.285120,-107.549334,29.092212,-107.190732
50%,2500.500000,125.000000,1376.110000,535.115000,594.915000,176.415000,55.555000,2686.600000,1208.630000,225.000000,0.000000,0.000000,33.536276,-94.505503,33.413396,-94.278565
75%,3750.250000,138.000000,1935.357500,761.000000,839.625000,238.685000,79.187500,3815.952500,1956.075000,238.000000,1.000000,1.000000,37.825708,-82.137262,37.779266,-82.254098
max,5000.000000,150.000000,2499.910000,1552.810000,1489.210000,299.970000,100.000000,6208.530000,4160.260000,250.000000,3.000000,3.000000,41.999318,-70.008170,41.999345,-70.004045


In [12]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Classification pipeline
clf_logreg = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(max_iter=1000))
    ]
)

# Train
clf_logreg.fit(X_train_c, y_train_c)

# Predict
y_pred_logreg = clf_logreg.predict(X_test_c)

# Evaluate
print("Logistic Regression Results:\n")
print(classification_report(y_test_c, y_pred_logreg))
print("Confusion Matrix:")
print(confusion_matrix(y_test_c, y_pred_logreg))

Logistic Regression Results:

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       900
           1       0.00      0.00      0.00       100

    accuracy                           0.90      1000
   macro avg       0.45      0.50      0.47      1000
weighted avg       0.81      0.90      0.85      1000

Confusion Matrix:
[[900   0]
 [100   0]]


c:\Users\ogbec\OneDrive\Documents\ANLT202\ai-fleet-demand-metroMove\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ogbec\OneDrive\Documents\ANLT202\ai-fleet-demand-metroMove\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ogbec\OneDrive\Documents\ANLT202\ai-fleet-demand-metroMove\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest classification pipeline
clf_rf = Pipeline(
    steps=[
        # Tree models don't strictly need scaling, so we skip StandardScaler here
        ("model", RandomForestClassifier(
            n_estimators=100,
            random_state=42
        ))
    ]
)

# Train the model
clf_rf.fit(X_train_c, y_train_c)

# Predict on test set
y_pred_rf = clf_rf.predict(X_test_c)

# Evaluate
print("=== Random Forest Results ===\n")
print(classification_report(y_test_c, y_pred_rf))
print("Confusion Matrix:")
print(confusion_matrix(y_test_c, y_pred_rf))

=== Random Forest Results ===

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       900
           1       0.00      0.00      0.00       100

    accuracy                           0.90      1000
   macro avg       0.45      0.50      0.47      1000
weighted avg       0.81      0.90      0.85      1000

Confusion Matrix:
[[900   0]
 [100   0]]


c:\Users\ogbec\OneDrive\Documents\ANLT202\ai-fleet-demand-metroMove\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ogbec\OneDrive\Documents\ANLT202\ai-fleet-demand-metroMove\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ogbec\OneDrive\Documents\ANLT202\ai-fleet-demand-metroMove\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

In [14]:
clf_logreg_bal = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(max_iter=1000, class_weight="balanced"))
    ]
)

clf_logreg_bal.fit(X_train_c, y_train_c)

y_pred_logreg_bal = clf_logreg_bal.predict(X_test_c)

print("=== Balanced Logistic Regression Results ===\n")
print(classification_report(y_test_c, y_pred_logreg_bal))
print(confusion_matrix(y_test_c, y_pred_logreg_bal))

=== Balanced Logistic Regression Results ===

              precision    recall  f1-score   support

           0       0.89      0.50      0.64       900
           1       0.09      0.46      0.16       100

    accuracy                           0.50      1000
   macro avg       0.49      0.48      0.40      1000
weighted avg       0.81      0.50      0.60      1000

[[454 446]
 [ 54  46]]


In [15]:
clf_rf_bal = Pipeline(
    steps=[
        ("model", RandomForestClassifier(
            n_estimators=200,
            random_state=42,
            class_weight="balanced"
        ))
    ]
)

clf_rf_bal.fit(X_train_c, y_train_c)

y_pred_rf_bal = clf_rf_bal.predict(X_test_c)

print("=== Balanced Random Forest Results ===\n")
print(classification_report(y_test_c, y_pred_rf_bal))
print(confusion_matrix(y_test_c, y_pred_rf_bal))

=== Balanced Random Forest Results ===

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       900
           1       0.00      0.00      0.00       100

    accuracy                           0.90      1000
   macro avg       0.45      0.50      0.47      1000
weighted avg       0.81      0.90      0.85      1000

[[900   0]
 [100   0]]


c:\Users\ogbec\OneDrive\Documents\ANLT202\ai-fleet-demand-metroMove\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ogbec\OneDrive\Documents\ANLT202\ai-fleet-demand-metroMove\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ogbec\OneDrive\Documents\ANLT202\ai-fleet-demand-metroMove\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

In [11]:
print("Classification Train:", X_train_c.shape, "Test:", X_test_c.shape)

Classification Train: (4000, 14) Test: (1000, 14)
